---

# Main Script to Run

---

## Steps

1. Open a Paper on Zotero
2. Copy the paper into a text file named "paper.txt". Save this file into a folder named after the author. (ex. Doe et al., 2015)
3. Create annotations, extract them into a text file, and save them as "notes.txt"
4. Drag & drop the folders into the "_inbox" folder, in the same parent folder where all the paper folders are stored. The folders in the _inbox folder will be the ones that the script finds and acts on.
5. Run the Script 
6. A set of Anki flashcards will be generated

## complete example

In [9]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Retrieve your OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the path to your _inbox folder
source_path = r"D:\OneDrive\_Carnegie Mellon (CMU)\60 Academic\63 Literature Review\63.002 Literature Review Exports from Zotero\63.003 Transparent Devices"


# Update these paths as needed
INBOX_PATH = source_path  # Ensure this variable is defined appropriately
ANKICONNECT_URL = "http://127.0.0.1:8765"
OPENAI_URL = "https://api.openai.com/v1/chat/completions"

# Parent deck name
PARENT_DECK = "CMU.63 Literature Review::63.010 test"
# PARENT_DECK = "CMU.63 Literature Review::63.002 Nano-Scale Bioelectronics"

In [10]:
# code

def create_deck(deck_name):
    """
    Creates a deck in Anki using AnkiConnect.
    """
    payload = {
        "action": "createDeck",
        "version": 6,
        "params": {"deck": deck_name}
    }
    response = requests.post(ANKICONNECT_URL, json=payload)
    return response.json()


def add_card_to_anki(deck_name, front, back):
    """
    Ensures the deck exists, then adds a Basic card to it.
    """
    create_deck(deck_name)
    payload = {
        "action": "addNotes",
        "version": 6,
        "params": {
            "notes": [
                {
                    "deckName": deck_name,
                    "modelName": "Basic",
                    "fields": {"Front": front, "Back": back},
                    "tags": ["paper", "notecard"]
                }
            ]
        }
    }
    response = requests.post(ANKICONNECT_URL, json=payload)
    return response.json()


def generate_notecards(notes_text):
    """
    Sends the content of notes.txt to the OpenAI Chat Completions endpoint
    and returns the raw text containing Q&A pairs formatted as notecards.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    prompt_messages = [
        {
            "role": "system",
            "content": (
                "You are given annotations from a research paper. "
                "Create concise question-answer pairs as notecards. "
                "Each notecard should start with 'Q:' for the question and 'A:' for the answer. "
                "Include the reference in the question (e.g., (Lastname et al., 2023)). "
                "Generate enough cards to cover the notes, but keep the total no greater than 10-15 cards."
            )
        },
        {"role": "user", "content": notes_text}
    ]
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": prompt_messages,
        "temperature": 0.7
    }
    response = requests.post(OPENAI_URL, headers=headers, json=payload)
    data = response.json()
    return data["choices"][0]["message"]["content"]


def parse_q_and_a(generated_text):
    """
    Parses the generated text into a list of (question, answer) tuples.
    Expects the format:
      Q: Question text
      A: Answer text
    """
    lines = generated_text.splitlines()
    flashcards = []
    question = ""
    answer = ""
    for line in lines:
        line = line.strip()
        if line.startswith("Q:"):
            if question and answer:
                flashcards.append((question, answer))
                question, answer = "", ""
            question = line[2:].strip()
        elif line.startswith("A:"):
            answer = line[2:].strip()
    if question and answer:
        flashcards.append((question, answer))
    return flashcards


def get_existing_decks():
    """
    Retrieves the list of existing decks from Anki using AnkiConnect.
    """
    payload = {"action": "deckNames", "version": 6}
    response = requests.post(ANKICONNECT_URL, json=payload)
    return response.json().get("result", [])


def main():
    if not os.path.exists(INBOX_PATH):
        print(f"Inbox path does not exist: {INBOX_PATH}")
        return

    # Get the current list of decks from Anki
    existing_decks = set(get_existing_decks())

    # List all folders in the _inbox
    folder_names = [
        folder for folder in os.listdir(INBOX_PATH)
        if os.path.isdir(os.path.join(INBOX_PATH, folder))
    ]

    for folder in folder_names:
        # Create the full deck name using the parent deck and the folder name
        full_deck_name = f"{PARENT_DECK}::{folder}"
        if full_deck_name in existing_decks:
            print(f"Deck '{full_deck_name}' already exists. Skipping folder '{folder}'.")
            continue

        folder_path = os.path.join(INBOX_PATH, folder)
        notes_file = os.path.join(folder_path, "notes.txt")
        paper_file = os.path.join(folder_path, "paper.txt")

        if not (os.path.exists(notes_file) and os.path.exists(paper_file)):
            print(f"Missing notes.txt or paper.txt in folder: {folder}")
            continue

        with open(notes_file, "r", encoding="utf-8") as nf:
            notes_text = nf.read()

        # Generate notecards text via OpenAI and parse the output
        raw_notecards_text = generate_notecards(notes_text)
        notecards = parse_q_and_a(raw_notecards_text)

        for question, answer in notecards:
            result = add_card_to_anki(full_deck_name, question, answer)
            print(f"Added notecard to deck '{full_deck_name}': {result}")


if __name__ == "__main__":
    main()


Deck 'CMU.63 Literature Review::63.010 test::Dijk et al., 2022' already exists. Skipping folder 'Dijk et al., 2022'.
Deck 'CMU.63 Literature Review::63.010 test::Khaldi et al., 2016' already exists. Skipping folder 'Khaldi et al., 2016'.
Deck 'CMU.63 Literature Review::63.010 test::Khodagholy et al., 2011' already exists. Skipping folder 'Khodagholy et al., 2011'.
Deck 'CMU.63 Literature Review::63.010 test::Middya et al., 2021' already exists. Skipping folder 'Middya et al., 2021'.
Deck 'CMU.63 Literature Review::63.010 test::Middya et al., 2025' already exists. Skipping folder 'Middya et al., 2025'.
Deck 'CMU.63 Literature Review::63.010 test::Sessolo et al., 2013' already exists. Skipping folder 'Sessolo et al., 2013'.
